In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import seaborn as sns
#from itertools import izip

In [3]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [4]:
#train_data.groupby('hospital_death').hist()
#list_plot = list(np.arange(169,173))

In [5]:
# df = train_data.iloc[:,list_plot]
# df['hospital_death'] = train_data['hospital_death']

In [6]:
#df.groupby('hospital_death').hist()
#train_data[['height','hospital_death']].groupby('hospital_death').hist()

In [7]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [8]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [9]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

#to_label_encode = list(data_dict[data_dict['Data Type'].isin(['string'])]['Variable Name'])

In [10]:
cat_var = cat_var + ['weightclass']

In [11]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [12]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [13]:
x_test = test_data.drop('hospital_death',axis=1)

In [14]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
x_comb.head()

In [15]:
list_drop = ['hospital_id','icu_id','encounter_id', 'patient_id', 'readmission_status']

In [16]:
# # Remove Features with more than 70 percent missing values
#x_comb = x_comb.drop(['readmission_status'],axis = 1)
x_comb = x_comb.drop(list_drop,axis = 1)

In [17]:
x_comb['hospital_admit_source'] = x_comb['hospital_admit_source'].replace({'Other ICU': 'ICU',
                                                                           'ICU to SDU':'SDU', 
                                                                           'Step-Down Unit (SDU)': 'SDU',        
                                                                           'Other Hospital':'Other',
                                                                           'Observation': 'Recovery Room',
                                                                           'Acute Care/Floor': 'Acute Care'})

In [18]:
x_comb['icu_type'] = x_comb['icu_type'].replace({'CCU-CTICU': 
                                                 'Grpd_CICU', 
                                                 'CTICU':'Grpd_CICU', 
                                                 'Cardiac ICU':'Grpd_CICU'})


In [19]:
x_comb['apache_2_bodysystem'] = x_comb['apache_2_bodysystem'].replace({'Undefined diagnoses': 
                                                                       'Undefined Diagnoses'})


In [20]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] =  x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [21]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [22]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))


100%|██████████| 28/28 [00:03<00:00,  7.75it/s]


In [23]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 72358.27it/s]


In [24]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [25]:
list_cols_nans = list(data_dict[data_dict['Category'].isin(['vitals','labs','labs blood gas'])]['Variable Name'])

In [26]:
list_cols_nans = list(set(list_cols_nans).intersection(set(x_comb.columns)))
#list_cols_nans

In [27]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

28it [00:01, 17.54it/s]


In [28]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'labs blood gas']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [29]:
binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10 ,labels=False))
    binned_cols.append(cols+'_binned')
    

100%|██████████| 16/16 [00:00<00:00, 39.70it/s]


In [30]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [31]:
featured_cols = binned_cols + ['age_category','height'+'_binned']

In [32]:
#2way interaction
bias_cols = ['gender','age']

In [33]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

100%|██████████| 2/2 [00:04<00:00,  2.12s/it]


In [34]:
#Critical -3way interaction


x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)
x_comb['age'+'gender'+'ethnicity' +'weighted_classt' ] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str) +'_' + x_comb['weightclass'].astype(str)


inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt'] +['age'+'gender'+'ethnicity'] + ['age'+'gender'+'ethnicity'+'weighted_classt']

In [35]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

100%|██████████| 35/35 [00:02<00:00, 15.44it/s]


In [36]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']


In [38]:
inter_cols1 = []

for col in tqdm(list_illness) :
    #x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    #inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

100%|██████████| 8/8 [00:03<00:00,  2.17it/s]


In [39]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 24/24 [00:01<00:00, 20.26it/s]


In [40]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

100%|██████████| 8/8 [00:01<00:00,  6.20it/s]


In [41]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 8/8 [00:00<00:00, 35.77it/s]


In [42]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']


In [43]:
# cat_count_var = []

# for i,col in tqdm(enumerate(new_cat_var)):
#     mean = x_comb['apache_4a_hospital_death_prob'].mean()
    
#     #Compute the number of values and the mean of each group
#     agg = x_comb.groupby(col)['apache_4a_hospital_death_prob'].agg(['count', 'mean'])
#     counts = agg['count']
#     means =  agg['mean']
    
#     #Compute the "smoothed" means
#     m= 3
#     smooth = (counts * means + m * mean) / (counts + m)
    
#     #Final_val
#     x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
#     cat_count_var.append(str(col)+'count_new')
    

In [44]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

28it [00:00, 57.51it/s]


In [45]:
# aggs = {}
# aggs['apache_4a_hospital_death_prob'] = ['median', 'mean', 'max', 'min']
# aggs['apache_4a_icu_death_prob'] = ['median', 'mean', 'max', 'min']


In [46]:
new_cat_var = list(set(new_cat_var).intersection(x_comb.columns))

In [47]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [48]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [49]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 17/17 [00:04<00:00,  4.23it/s]


In [50]:
def mode(x):
    return Counter(x).most_common(1)[0][0]


In [51]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count',mode]
    

100%|██████████| 28/28 [00:00<00:00, 152125.02it/s]


In [52]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [53]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 28/28 [04:40<00:00, 10.01s/it]


In [ ]:
# x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

# le = LabelEncoder()
# x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [ ]:
#x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [54]:
x_comb.shape

(131021, 2573)

In [55]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [56]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [57]:
# random_state = 42
# np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':6,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':15,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.2, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [58]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [59]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    15000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=2500,
                    verbose_eval=200
                    )

Training until validation scores don't improve for 2500 rounds.
[200]	training's auc: 0.895622	valid_1's auc: 0.88506
[400]	training's auc: 0.905103	valid_1's auc: 0.891948
[600]	training's auc: 0.91124	valid_1's auc: 0.895767
[800]	training's auc: 0.91564	valid_1's auc: 0.898027
[1000]	training's auc: 0.919117	valid_1's auc: 0.899665
[1200]	training's auc: 0.922028	valid_1's auc: 0.900697
[1400]	training's auc: 0.924613	valid_1's auc: 0.901533
[1600]	training's auc: 0.926943	valid_1's auc: 0.902153
[1800]	training's auc: 0.92913	valid_1's auc: 0.902668
[2000]	training's auc: 0.931202	valid_1's auc: 0.903142
[2200]	training's auc: 0.933123	valid_1's auc: 0.903488
[2400]	training's auc: 0.934939	valid_1's auc: 0.903781
[2600]	training's auc: 0.936624	valid_1's auc: 0.903993
[2800]	training's auc: 0.938212	valid_1's auc: 0.904195
[3000]	training's auc: 0.939793	valid_1's auc: 0.904377
[3200]	training's auc: 0.941311	valid_1's auc: 0.904532
[3400]	training's auc: 0.94278	valid_1's auc: 0.

In [60]:
predictions = lgb_clf.predict(test_x)

In [61]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [62]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/sample_try_withouticuhosid.csv',index = False)


In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
list_unimp = list(feature_imp[feature_imp['Value'] == 0]['Feature'])

In [ ]:
feature_imp[feature_imp['Feature'] == 'map_apache']

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=130, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()
# #data_sub

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 20000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 1500)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions /10.0
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/fold10_seed_rs_lgb_24thfeb.csv',index = False)


In [ ]:
#feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
#                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
#feature_imp[feature_imp['Feature'] == 'weightclass']


In [ ]:
#feature_imp[feature_imp['Value'] == 0]

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/sub_10fold_keep_missing_as_it_is_18thFeb2020_newfeat.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_12thfeb.csv')

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4_lgbold_10thfeb_new_bmi_impute.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_10thFeb2020_lgb_type4_ff_20_na_bmi.csv')
# #a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb_type3_ff_27.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_11thFeb2020_lgb_type3_ff_27.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/pred_lgb_keeping_cat_cont_interchanged_11th_feb.csv')
# a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4tillnpw_11thfeb.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death']
           )/2.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_18thfeb.csv',index = False)

In [ ]:
# def hospital_type(x):
    
#     if pd.isna(x):
#         return np.nan
    
#     elif x <= 4:
#         return "less_icu"
    
#     elif x > 8:
#         return "more_icu"
#     else:
#         return "med_icu"
